### Set the path and enivronment

In [1]:
import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TZ'] = 'Aisa/Shanghai'

### Load functions

In [2]:
from itertools import product
from solver import train_resnet_with_cub

c:\Users\雪浪\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Hyper-parameters
Due to the computation resource restricted, we first train on a fixed small epoch to find the best conmbination of `fine_tuning_lrs` and `output_lrs`

In [3]:
# set hyper-parameters here
num_epochs = [20]
fine_tuning_lrs = [5e-5, 1e-4, 5e-4]
output_lrs = [1e-3, 2e-3, 5e-3, 8e-3, 1e-2]

### Train

In [ ]:
configurations = list(product(fine_tuning_lrs, output_lrs))
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(num_epochs, fine_tuning_lr=config[0], output_lr=config[1])
    best_accs.extend(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy_lr.txt', 'w') as f:
    f.write("Configuration        Accuracy\n")
    f.write("=" * 30 + "\n")
    for config, accuracy in zip(configurations, best_accs):
        f.write("({:>7.5f}, {:>7.5f})   {:>8.6f}\n".format(config[0], config[1], accuracy))

### Find the best epoch

In [1]:
!more best_accuracy_lr.txt

Configuration        Accuracy
(0.00005, 0.00100)   0.277356
(0.00005, 0.00200)   0.419399
(0.00005, 0.00500)   0.559199
(0.00005, 0.00800)   0.582327
(0.00005, 0.01000)   0.599413
(0.00010, 0.00100)   0.297549
(0.00010, 0.00200)   0.453227
(0.00010, 0.00500)   0.572316
(0.00010, 0.00800)   0.605281
(0.00010, 0.01000)   0.615119
(0.00050, 0.00100)   0.447705
(0.00050, 0.00200)   0.556438
(0.00050, 0.00500)   0.639972
(0.00050, 0.00800)   0.652399
(0.00050, 0.01000)   0.652054


We can find that the best configuration of the model after training **10** epochs is $(0.01, 0.05)$. Now we use this configuration to find the best epoch.

In [ ]:
num_epochs = [10 * (c + 1) for c in range(10)]
fine_tuning_lrs = [5e-4]
output_lrs = [8e-3]

# init the configurations
configurations = list(product(fine_tuning_lrs, output_lrs))
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(num_epochs, fine_tuning_lr=config[0], output_lr=config[1])
    best_accs.extend(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy_ep.txt', 'w') as f:
    f.write("Epoch  Accuracy\n")
    f.write("=" * 50 + "\n")
    for num_epoch, accuracy in zip(num_epochs, best_accs):
        f.write("{:>2}     {:>8.6f}\n".format(num_epoch, accuracy))

In [ ]:
!more best_accuracy_ep.txt